<a href="https://colab.research.google.com/github/kmalcok/TSL_Study/blob/main/TSL_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kaggle API anahtarını yükleme
from google.colab import files
files.upload()  # Bu, bilgisayarınızdan kaggle.json dosyasını yüklemenizi sağlayacak

# Kaggle API anahtarını yapılandırma
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Kaggle CLI yükleme
!pip install kaggle

# Veri setini indirme (örnek: titanic veri seti)
!kaggle datasets download -d berkaykocaoglu/tr-sign-language

# İndirilen zip dosyasını açma
import zipfile
with zipfile.ZipFile('tr-sign-language.zip', 'r') as zip_ref:
    zip_ref.extractall('tr-sign-language')




In [ ]:
import os

train_dir= "/content/tr-sign-language/tr_signLanguage_dataset/train"
test_dir= "/content/tr-sign-language/tr_signLanguage_dataset/test"

print("train classes:" , os.listdir(train_dir))
print("test classes:" , os.listdir(test_dir))

def preprocess_data(image)

def label_data(etiket)

def load_data(data_type, path):



train classes: ['C', 'R', 'P', 'J', 'K', 'O', 'N', 'I', 'Y', 'M', 'H', 'E', 'T', 'U', 'del', 'A', 'L', 'S', 'D', 'Z', 'F', 'G', 'B', 'space', 'V', 'nothing']
test classes: ['C', 'R', 'P', 'J', 'K', 'O', 'N', 'I', 'Y', 'M', 'H', 'E', 'T', 'U', 'del', 'A', 'L', 'S', 'D', 'Z', 'F', 'G', 'B', 'space', 'V', 'nothing']


In [ ]:
from google.colab import drive
import shutil
# Colab'de bulunan dosyanın yolu
source_path = '/content/tr-sign-language'

# Google Drive'daki hedef dizin
destination_path = '/content/drive/MyDrive/tr-sign-language'

# Hedef dizin mevcutsa silme
if os.path.exists(destination_path):
    shutil.rmtree(destination_path)
# Dosyayı kopyalama
shutil.copytree(source_path, destination_path)

print("Dosya başarıyla kopyalandı.")

Dosya başarıyla kopyalandı.


In [ ]:
import os
import glob
import unittest
from PIL import Image, ImageOps
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.applications import VGG16, MobileNet, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import keras_tuner as kt
import tensorflow as tf

def load_data(data_path):
    train_path = os.path.join(data_path, 'train')
    test_path = os.path.join(data_path, 'test')

    def load_images_from_folder(folder):
        images = []
        labels = []
        subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]

        def process_image_file(image_file, label):
            image = Image.open(image_file).convert('RGB')
            image = pad_image_to_target_size(image, (224, 224))
            image = normalize_image(np.array(image))
            return image, label

        with ThreadPoolExecutor() as executor:
            futures = []
            for subfolder in subfolders:
                label = os.path.basename(subfolder)
                image_files = glob.glob(os.path.join(subfolder, '*.jpg'))
                for image_file in image_files:
                    futures.append(executor.submit(process_image_file, image_file, label))

            for future in futures:
                image, label = future.result()
                images.append(image)
                labels.append(label)

        return images, labels

    def pad_image_to_target_size(image, target_size):
        desired_width, desired_height = target_size
        width, height = image.size
        if width == desired_width and height == desired_height:
            return image
        delta_width = desired_width - width
        delta_height = desired_height - height
        padding = (delta_width // 2, delta_height // 2, delta_width - (delta_width // 2), delta_height - (delta_height // 2))
        new_image = ImageOps.expand(image, padding)
        return new_image

    def normalize_image(image):
        return image / 255.0

    train_images, train_labels = load_images_from_folder(train_path)
    test_images, test_labels = load_images_from_folder(test_path)

    return (np.array(train_images), np.array(train_labels)), (np.array(test_images), np.array(test_labels))

data_path = '/content/drive/MyDrive/tr-sign-language/tr_signLanguage_dataset'
(train_images, train_labels), (test_images, test_labels) = load_data(data_path)

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(
    train_images, train_labels, batch_size=32
)

val_generator = val_datagen.flow(
    val_images, val_labels, batch_size=32
)

input_shape = (224, 224, 3)
num_classes = len(np.unique(train_labels))

def build_model(base_model, hp):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(hp.Int('units_1', min_value=512, max_value=1024, step=128), activation='relu')(x)
    x = Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1))(x)
    x = Dense(hp.Int('units_2', min_value=256, max_value=512, step=128), activation='relu')(x)
    x = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def model_builder(hp, base_model):
    model = build_model(base_model, hp)
    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def tune_model(base_model, project_name):
    tuner = kt.Hyperband(
        lambda hp: model_builder(hp, base_model),
        objective='val_accuracy',
        max_epochs=20,
        factor=3,
        directory='my_dir',
        project_name=project_name
    )

    stop_early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    tuner.search(train_generator, epochs=50, validation_data=val_generator, callbacks=[stop_early])

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)
    return model

vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
mobilenet_base = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

vgg16_model = tune_model(vgg16_base, 'vgg16')
mobilenet_model = tune_model(mobilenet_base, 'mobilenet')
inceptionv3_model = tune_model(inceptionv3_base, 'inceptionv3')

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]

history_vgg16 = vgg16_model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=callbacks)
history_mobilenet = mobilenet_model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=callbacks)
history_inceptionv3 = inceptionv3_model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=callbacks)

vgg16_preds = vgg16_model.predict(test_images)
mobilenet_preds = mobilenet_model.predict(test_images)
inceptionv3_preds = inceptionv3_model.predict(test_images)

final_preds = (vgg16_preds + mobilenet_preds + inceptionv3_preds) / 3

final_labels = np.argmax(final_preds, axis=1)
test_labels_argmax = np.argmax(test_labels, axis=1)
ensemble_accuracy = accuracy_score(test_labels_argmax, final_labels)

print(f'Ensemble Model Accuracy: {ensemble_accuracy:.2f}')

class TestModels(unittest.TestCase):
    def setUp(self):
        self.input_shape = (224, 224, 3)
        self.num_classes = len(np.unique(train_labels))

    def test_vgg16_model(self):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.input_shape)
        model = tune_model(base_model, 'vgg16')
        self.assertIsInstance(model, Model)
        self.assertEqual(model.output_shape[-1], self.num_classes)

    def test_mobilenet_model(self):
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=self.input_shape)
        model = tune_model(base_model, 'mobilenet')
        self.assertIsInstance(model, Model)
        self.assertEqual(model.output_shape[-1], self.num_classes)

    def test_inceptionv3_model(self):
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=self.input_shape)
        model = tune_model(base_model, 'inceptionv3')
        self.assertIsInstance(model, Model)
        self.assertEqual(model.output_shape[-1], self.num_classes)

    def test_ensemble_accuracy(self):
        vgg16_preds = vgg16_model.predict(test_images)
        mobilenet_preds = mobilenet_model.predict(test_images)
        inceptionv3_preds = inceptionv3_model.predict(test_images)
        final_preds = (vgg16_preds + mobilenet_preds + inceptionv3_preds) / 3
        final_labels = np.argmax(final_preds, axis=1)
        test_labels_argmax = np.argmax(test_labels, axis=1)
        ensemble_accuracy = accuracy_score(test_labels_argmax, final_labels)
        self.assertGreater(ensemble_accuracy, 0.8)  # Example threshold

if __name__ == '__main__':
    unittest.main()
